<a href="https://colab.research.google.com/github/jellyfizhi/genAI_experiment/blob/main/genAIExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
save_path="/llama3"
# add your hugging face token here in the form hf_token = "xxxxxxxxxx"
def download_LLM(model_name, save_path, hf_token):
    """
    Downloads a LLM model from Hugging Face and saves it locally.

    Parameters:
        model_name (str): The name of the llama-3-8b model to download. Options:
                          - 'llama3' (small)
                          - 'llama3-medium'
                          - 'llama3-large'
                          - 'llama3-xl'
        save_path (str): Directory to save the model and tokenizer.
    """
    snapshot_path = snapshot_download(
    repo_id=model_name,
    token=hf_token,
    local_dir=save_path
    )

# print(f"Private model snapshot downloaded to: {snapshot_path}")

    print(f"✅ LLM model ('{model_name}') downloaded and saved to '{save_path}'.")

# Example usage
#download_LLM(model_name, save_path, hf_token)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3405: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from accelerate import load_checkpoint_and_dispatch

# Clear GPU memory before loading
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()  # Collect any lingering memory fragments
        print(f"GPU Memory Cleared. Free: {torch.cuda.memory_reserved() / 1024**3:.2f} GiB")
# Install required libraries if not already installed:
# pip install bitsandbytes accelerate

# Load the model and tokenizer with quantization
def load_LLM(save_path):
    """
    Loads the model and tokenizer from a saved local path with quantization.
    """
    # Clear GPU memory before loading
    clear_gpu_memory()

    # Define 4-bit quantization configuration
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Enable 4-bit quantization (lower than 8-bit)

        bnb_4bit_quant_type="nf4",  # Use NF4 quantization (optimized for LLMs)
        bnb_4bit_use_double_quant=True,  # Double quantization for extra savings
        bnb_4bit_compute_dtype="float16",  # Compute in FP16
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(save_path)

    # Load model with 4-bit quantization
    model = AutoModelForCausalLM.from_pretrained(
        save_path,
        quantization_config=quantization_config,  # Pass the 4-bit quantization config
        device_map="auto",  # Automatically split across GPU/CPU
        low_cpu_mem_usage=True,  # Minimize CPU memory spikes
    )

    # Optional: Offload to CPU/disk if GPU memory is still insufficient
 #   model = load_checkpoint_and_dispatch(
 #       model,
 #       checkpoint=save_path,
 #       device_map="auto",
 #       offload_folder="./offload",  # Directory for offloaded weights
 #   )

    return tokenizer, model

# Function to chat with LLMs
def chat_with_LLM(save_path):
    # Load the model and tokenizer
    tokenizer, model = load_LLM(save_path)  # Adjust save_path as needed

    print("🤖 Chatbot: Hello! Type 'exit' to end the chat.")

    while True:
        user_input = input("🧑 You: ")
        if user_input.lower() == "exit":
            print("🤖 Chatbot: Goodbye!")
            break

        # Tokenize input and move to GPU if available
        input_ids = tokenizer.encode(user_input, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

        # Generate a response
        output = model.generate(
            input_ids,
            max_length=100,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,  # Optional: Adds variety to responses
            top_k=50,        # Optional: Improves response quality
            top_p=0.95       # Optional: Improves response quality
        )

        # Decode and print response
        response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
        print(f"🤖 Chatbot: {response}")

In [ ]:
# Run the chatbot
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
save_path="/llama3"
chat_with_LLM(save_path)